## 17/12/2020 Update
# Linking and Comparing Local Connect Stats with Low-rank Recurrent Network
* From Local Connect Stats cite[Sharpee]
* From Low-rank structure to EI 

In [1]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import numpy as np
import matplotlib.pylab as plt
import matplotlib
%matplotlib qt
from numpy import linalg as la
from scipy.optimize import fsolve
from scipy import linalg as scpla
import seaborn as sb
from mpl_toolkits.axes_grid1 import make_axes_locatable
from cmath import *
from mpl_toolkits.mplot3d import Axes3D
from scipy.optimize import fsolve,leastsq 
from math import tanh,cosh
import math
from sympy import *
extras_require = {'PLOT':['matplotlib>=1.1.1,<3.0']},

## Check the results in Sharpee's paper
* local stats, EE,EI,IE,II occupy their own properties, independently
* no pair correlation


In [178]:
Nt=np.array([2000,1000])
NE,NI=Nt[0],Nt[1]
at = Nt/N
N=np.sum(Nt)
aee,aei,aie,aii=4.5,-6.0,5.0,-2.5
JEEsub,JEIsub,JIEsub,JIIsub=aee*np.ones((NE,NE))/NE,aei*np.ones((NE,NI))/NI,aie*np.ones((NI,NE))/NE,aii*np.ones((NI,NI))/NI
J = np.zeros((NE+NI,NE+NI))

# determine gcd
Gmat=np.zeros((2,2))
Gmatorg=np.array([[0.8,1.2],[0.9,0.2]])
# Gmat/=np.sqrt(N)
for i in range(2):
    for j in range(2):
        Gmat[i,j]=Gmatorg[i,j]/np.sqrt(N)
# EE
gij=Gmat[0,0]*np.random.randn(NE,NE)
J[:NE,:NE]=gij.copy()
# EI
gij=Gmat[0,1]*np.random.randn(NE,NI)
J[:NE,NE:]=gij.copy()
# IE
gij=Gmat[1,0]*np.random.randn(NI,NE)
J[NE:,:NE]=gij.copy()
# II
gij=Gmat[1,1]*np.random.randn(NI,NI)
J[NE:,NE:]=gij.copy()

P = np.zeros((N,N))
P[:NE,:NE],P[:NE,NE:],P[NE:,:NE],P[NE:,NE:]=JEEsub.copy(),JEIsub.copy(),JIEsub.copy(),JIIsub.copy()
X = P.copy()+J.copy()

In [113]:
gavg=np.sqrt(np.sum(np.sum(Gmat**2)))/2.0
print('averaging g noise:',gavg*np.sqrt(N))

averaging g noise: 0.8558621384311844


In [105]:
aee,aei,aie,aii=4.5,-6.0,5.0,-2.5
Asmall=np.array([[aee,aei],[aie,aii]])
eigva,_=la.eig(Asmall)
_,sigva,_=la.svd(Asmall)
print('eigenvalue:',eigva,'singular value:',sigva,'eigG',np.sqrt(eigG[0]))

eigenvalue: [1.+4.21307489j 1.-4.21307489j] singular value: [9.12570385 2.05463604] eigG 0.8546614831429354


In [47]:
# calculate singular value circles
_,sigvrand,_=la.svd(J)
fig,ax=plt.subplots(figsize=(6,6))
n, bins, patches = ax.hist(sigvrand, 100, density=True, alpha=0.5)
# ax.scatter(np.real(eigvrand),np.imag(eigvrand),s=5.0,c='blue',alpha=0.5)

In [179]:
# all about calculate eigenvalues
eigvrand,eigvecrand=la.eig(J)
eigstruct,eigvecstruct=la.eig(P)
# summation
X = P.copy()+J.copy()
eigvx,eigvecx=la.eig(X)
# theoretical
M = np.zeros((2,2))
for i in range(2):
    for j in range(2):
        M[i,j]=(Gmat[i,j]*np.sqrt(N))**2*(Nt[j]/N)
eigG,eigvecG=la.eig(M)
# low rank small
Asmall=np.array([[aee,aei],[aie,aii]])
eigva,_=la.eig(Asmall)


In [180]:
ticks = np.linspace(-2.0,2.0,3)
lims= [-2.1,2.1]

fig,ax=plt.subplots(figsize=(6,6))
ax.plot(np.sqrt(eigG[0])*np.ones(100),np.linspace(-1.0,1.0,100))
ax.scatter(np.real(eigvrand),np.imag(eigvrand),s=5.0,c='blue',alpha=0.5)
ax.scatter(np.real(eigstruct[:4]),np.zeros(4),s=15.0,c='red',alpha=0.5)
# ax.set_xlim(lims)
# ax.set_ylim(lims)
# ax.set_xticks(ticks)
# ax.set_yticks(ticks)
ax.set_aspect('equal')
ax.grid()

## relate this to low rank (here rank-2)

In [109]:
lvecX,svX,rvecXh=la.svd(X)
rvecX=rvecXh.T
m=lvecX[:,:2].copy()
n=rvecX[:,:2].copy()
n[:,0]*=svX[0]
n[:,1]*=svX[1]

# n[:,1]=rvecX[:,2]*svX[2]
# m[:,1]=lvecX[:,2]

In [110]:
print('singular value:',svX[:4])

singular value: [9.21533954 2.36599547 1.84315887 1.83814427]


In [111]:
morg,norg=m,n
fig,ax=plt.subplots(4,4,figsize=(9,9))
for i in range(2):
    for j in range(2):
        ax[i][j].scatter(morg[:NE,i],morg[:NE,j],s=5.0,c='red',alpha=0.05)
        ax[i][j].scatter(morg[NE:,i],morg[NE:,j],s=5.0,c='blue',alpha=0.05)
for i in range(2):
    for j in range(2):
        ax[2+i][2+j].scatter(norg[:NE,i],norg[:NE,j],s=5.0,c='red',alpha=0.05)
        ax[2+i][2+j].scatter(norg[NE:,i],norg[NE:,j],s=5.0,c='blue',alpha=0.05)
        
for i in range(2):
    for j in range(2):
        ax[i][2+j].scatter(morg[:NE,i],norg[:NE,j],s=5.0,c='red',alpha=0.05)
        ax[i][2+j].scatter(morg[NE:,i],norg[NE:,j],s=5.0,c='blue',alpha=0.05)
for i in range(2):
    for j in range(2):
        ax[2+i][j].scatter(norg[:NE,i],morg[:NE,j],s=5.0,c='red',alpha=0.05)
        ax[2+i][j].scatter(norg[NE:,i],morg[NE:,j],s=5.0,c='blue',alpha=0.05)
for i in range(4):
    for j in range(4):
        # ax[i][j].set_xlim(lims)
        # ax[i][j].set_ylim(lims)
        # ax[i][j].set_xticks(ticks)
        # ax[i][j].set_yticks(ticks)
        ax[i][j].set_aspect('equal')
        ax[i][j].grid()

In [117]:
# test Dale's law
Nt=np.array([300,100])
NE,NI=Nt[0],Nt[1]
N=np.sum(Nt)
aee,aei,aie,aii=4.5,-6.0,5.0,-2.5
JEEsub,JEIsub,JIEsub,JIIsub=aee*np.ones((NE,NE))/NE,aei*np.ones((NE,NI))/np.sqrt(NE*NI),aie*np.ones((NI,NE))/np.sqrt(NE*NI),aii*np.ones((NI,NI))/NI
P = np.zeros((N,N))
P[:NE,:NE],P[:NE,NE:],P[NE:,:NE],P[NE:,NE:]=JEEsub.copy(),JEIsub.copy(),JIEsub.copy(),JIIsub.copy()
eigvP,_=la.eig(P)
_,sigvp,_=la.svd(P)
Asmall=np.array([[aee,aei],[aie,aii]])
eigva,_=la.eig(Asmall)
_,sigva,_=la.svd(Asmall)

print('eigenvalue:',eigvP[:2],eigva[:2])
print('singular value:',sigvp[:2],sigva[:2])

eigenvalue: [1.+4.21307489j 1.-4.21307489j] [1.+4.21307489j 1.-4.21307489j]
singular value: [9.12570385 2.05463604] [9.12570385 2.05463604]


In [184]:
# test Dale's law
Nt=np.array([100,1000])
NE,NI=Nt[0],Nt[1]
N=np.sum(Nt)
at=Nt/N
# determine gcd
Gmat=np.zeros((2,2))
Gmatorg=np.array([[0.8,1.2],[0.9,0.2]])
# Gmat/=np.sqrt(N)
for i in range(2):
    for j in range(2):
        Gmat[i,j]=Gmatorg[i,j]/np.sqrt(N)
# EE
J=np.zeros((N,N))

gij=np.zeros((NE,NE))
gij=Gmat[0,0]*np.random.randn(NE,NE)
J[:NE,:NE]=gij.copy()
# EI
gij=np.zeros((NE,NI))
gij=Gmat[0,1]*np.random.randn(NE,NI)
J[:NE,NE:]=gij.copy()
# IE
gij=np.zeros((NI,NE))
gij=Gmat[1,0]*np.random.randn(NI,NE)
J[NE:,:NE]=gij.copy()
# II
gij=np.zeros((NI,NI))
gij=Gmat[1,1]*np.random.randn(NI,NI)
J[NE:,NE:]=gij.copy()

gavg=0
for i in range(2):
    for j in range(2):
        gavg+=np.sqrt(Gmatorg[i,j]**2*at[i]*at[j])
M = np.zeros((2,2))
for i in range(2):
    for j in range(2):
        M[i,j]=(Gmatorg[i,j]**2)*at[j]

eigvj,_=la.eig(J)
eigvM,_=la.eig(M)
ticks = np.linspace(-2.0,2.0,3)
lims= [-2.1,2.1]

fig,ax=plt.subplots(figsize=(6,6))
ax.plot(np.sqrt(eigvM[0])*np.ones(100),np.linspace(-1.0,1.0,100))
ax.scatter(np.real(eigvj),np.imag(eigvj),s=5.0,c='blue',alpha=0.5)
print('average eigv:',gavg)


average eigv: 0.858253007850327


In [227]:
# testing distribution of J_{ij}
NE,NI=750,250
sigm,sign=0.5,1.2
mE,nE=sigm*np.random.randn(NE,1)/np.sqrt(NE),sign*np.random.randn(NE,1)/np.sqrt(NE)
JEE = mE@nE.T 
eigv,eigvec=la.eig(JEE)
fig,ax=plt.subplots(figsize=(6,6))
ax.scatter(np.real(eigvj),np.imag(eigvj),s=5.0,c='blue',alpha=0.5)

JEEoff=JEE.copy()
JEEdiag=np.zeros(NE)
for i in range(NE):
    JEEoff[i,i:NE-1]=JEEoff[i,i+1:]
    JEEdiag[i]=JEE[i,i]
fig,ax=plt.subplots(2,1,figsize=(6,6))
Joff=JEEoff[:,:NE-1]
Joff=np.reshape(Joff,(NE**2-NE,1))
n, bins, patches = ax[0].hist(Joff, 100, density=True, alpha=0.5)
n, bins, patches = ax[1].hist(JEEdiag, 100, density=True, alpha=0.5)
print(sigm*sign)



0.6


## What's the distribution of $J_{i,j}$ in $\tilde{\mathbf{X}}=\mathbf{X}+\mathbf{p}$

In [229]:
# testing distribution of J_{ij} in \tilde{X}
NE,NI=2000,250
sigm,sign,grandom=0.6,1.0,0.5
rho=0.0
zetax,zetay=np.random.randn(NE,1),np.random.randn(NE,1)
m = sigm*zetax/np.sqrt(NE)
n =sign*(rho*zetax+np.sqrt(1-rho**2)*zetay)/np.sqrt(NE)
P = m@n.T 
X = (grandom)*np.random.randn(NE,NE)/np.sqrt(NE)

Xtilde=X+P

eigv,eigvec=la.eig(Xtilde)
fig,ax=plt.subplots(figsize=(6,6))
ax.scatter(np.real(eigv),np.imag(eigv),s=5.0,c='blue',alpha=0.5)

JEEoff=Xtilde.copy()
JEEdiag=np.zeros(NE)
for i in range(NE):
    JEEoff[i,i:NE-1]=JEEoff[i,i+1:]
    JEEdiag[i]=Xtilde[i,i]
fig,ax=plt.subplots(2,1,figsize=(6,6))
Joff=JEEoff[:,:NE-1]
Joff=np.reshape(Joff,(NE**2-NE,1))
n, bins, patches = ax[0].hist(Joff, 100, density=True, alpha=0.5)
n, bins, patches = ax[1].hist(JEEdiag, 100, density=True, alpha=0.5)

## Generate Random Block Matrix, embedding low rank structur $\tilde{\mathbf{X}}_{\mathrm{type}}=\mathbf{X}_{\mathrm{type}}+\mathbf{P}_{\mathrm{type}}$

In [237]:
Nt=np.array([1000,1000])
NE,NI=Nt[0],Nt[1]
at = Nt/N
N=np.sum(Nt)
xee,xei,xie,xii=4.5,6.0,4.0,2.5
# subject to
sigmE,signI,sigmI,signE=2.0,3.0,1.0,4.0
rhoE=xee/sigmE/signE
rhoI=xii/sigmI/signI
X,P=np.random.randn(N,N),np.zeros((N,N))
X[:NE,:NE],X[:NE,NE:]=xee*X[:NE,:NE]/np.sqrt(N),xei*X[:NE,NE:]/np.sqrt(N)
X[NE:,:NE],X[NE:,NE:]=xie*X[NE:,:NE]/np.sqrt(N),xii*X[NE:,NE:]/np.sqrt(N)
m,n=np.zeros((N,1)),np.zeros((N,1))
zetaxE,zetayE,zetaxI,zetayI=np.random.randn(NE),np.random.randn(NE),np.random.randn(NI),np.random.randn(NI)
m[:NE,0]=sigmE*zetaxE/np.sqrt(N)
n[:NE,0]=signE*(rhoE*zetaxE+np.sqrt(1-rhoE**2)*zetayE)/np.sqrt(N)
m[NE:,0]=sigmI*zetaxI/np.sqrt(N)
n[NE:,0]=signI*(rhoI*zetaxI+np.sqrt(1-rhoI**2)*zetayI)/np.sqrt(N)
P=m@n.T 
Xtilde=X.copy()+P.copy()

XEI,XTEI=X[:NE,NE:],Xtilde[:NE,NE:]
eigvEI,eigvec=la.eig(XEI)
eigvTEI,eigvec=la.eig(XTEI)
fig,ax=plt.subplots(1,2,figsize=(6,6))
ax[0].scatter(np.real(eigvEI),np.imag(eigvEI),s=5.0,c='blue',alpha=0.5)
ax[1].scatter(np.real(eigvTEI),np.imag(eigvTEI),s=5.0,c='blue',alpha=0.5)

XEE,XTEE=X[:NE,:NE],Xtilde[:NE,:NE]
eigvEE,eigvec=la.eig(XEE)
eigvTEE,eigvec=la.eig(XTEE)
fig,ax=plt.subplots(1,2,figsize=(6,6))
ax[0].scatter(np.real(eigvEE),np.imag(eigvEE),s=5.0,c='red',alpha=0.5)
ax[1].scatter(np.real(eigvTEE),np.imag(eigvTEE),s=5.0,c='red',alpha=0.5)

In [238]:
phiacttt=np.zeros((N,1))
phiacttt[:NE,0],phiacttt[NE:,0]=(rhoE*zetaxE+np.sqrt(1-rhoE**2)*zetayE)/np.sqrt(N),(rhoI*zetaxI+np.sqrt(1-rhoI**2)*zetayI)/np.sqrt(N)
eitatt=Xtilde@phiacttt
eitarand=X@phiacttt
coveitatt=eitatt@eitatt.T 
coveitarand=eitarand@eitarand.T 
deltaEt,deltaEr=coveitatt[:NE,:NE],coveitarand[:NE,:NE]
deltaIt,deltaIr=coveitatt[NE:,NE:],coveitarand[NE:,NE:]

In [240]:
print('std, Et:',np.std(deltaEt),'; random:',np.std(deltaEr))
print('std, It:',np.std(deltaIt),'; random:',np.std(deltaIr))

std, Et: 0.03849542094475085 ; random: 0.01399065517540076
std, It: 0.012372709451130663 ; random: 0.005562922293971781
